In [1]:
from fastcore.utils import *
from fasthtml.common import *
from fasthtml.jupyter import *
from fastlite import *
import fasthtml.components as fh
import httpx


# Fast HTML / Webdev fundamentals

In [2]:
daisy_hdrs = (
    Link(href='https://cdn.jsdelivr.net/npm/daisyui@5', rel='stylesheet', type='text/css'),
    Script(src='https://cdn.jsdelivr.net/npm/@tailwindcss/browser@4')
)


In [3]:
app = FastHTML(hdrs=daisy_hdrs)
app


In [4]:
rt = app.route


In [5]:

def get_preview(app): return partial(HTMX, app=app, host="localhost", port=8000)
p = get_preview(app)
p

functools.partial(<function HTMX at 0x115845c70>, app=<fasthtml.core.FastHTML object at 0x115739550>, host='localhost', port=8000)

In [6]:
p = get_preview(app)


In [7]:
c = P("Hello, World", cls='text-3xl')
c


```html
<p class="text-3xl">Hello, World</p>

```

In [8]:
p(c)


In [9]:
acc = Div(
        Div(
        Input(type='radio', name='my-accordion-3', checked='checked'),
        Div('How do I create an account?', cls='collapse-title font-semibold'),
        Div('Click the "Sign Up" button in the top right corner and follow the registration process.', cls='collapse-content text-sm'),
        cls='collapse collapse-plus bg-base-100 border border-base-300'
    ),
    Div(
        Input(type='radio', name='my-accordion-3'),
        Div('I forgot my password. What should I do?', cls='collapse-title font-semibold'),
        Div('Click on "Forgot Password" on the login page and follow the instructions sent to your email.', cls='collapse-content text-sm'),
        cls='collapse collapse-plus bg-base-100 border border-base-300'
    ),
    Div(
        Input(type='radio', name='my-accordion-3'),
        Div('How do I update my profile information?', cls='collapse-title font-semibold'),
        Div('Go to "My Account" settings and select "Edit Profile" to make changes.', cls='collapse-content text-sm'),
        cls='collapse collapse-plus bg-base-100 border border-base-300'
    )
)


In [10]:
p(acc)


In [11]:
c = Button('Default', cls="btn")
p(c)


In [12]:
def Button(*c, cls='', **kw):
    return fh.Button(*c, cls=f"btn {cls}", **kw)


In [13]:
c = Button('Default')
p(c)



In [14]:
@rt
def showbtn(): return c


In [15]:
@rt
def index():
    return Titled('Hello, Jupyter',
           P('Welcome to the FastHTML + Jupyter example'),
           Button('Click', hx_get='/click', hx_target='#dest'),
           Div(id='dest')
    )


In [16]:
server = JupyUvi(app)


In [17]:
@rt
def click(): return P('You clicked me!')


In [18]:
p('/showbtn')



In [19]:
def Input(*c, cls='', **kw): 
    return fh.Input(*c, cls=f'input {cls}', **kw)


In [20]:
@rt
def btnres(nm:str='Dad'): return f"Hi, {nm}"

In [21]:
btnres("ben")

'Hi, ben'

TO avoid typing stuff we don't need again, we replace the Input nameaspace with a wrapper funnction that calls the input class with out preferred args

In [22]:
p(Input(type='text', placeholder='Type here'))

In [23]:
@rt
def testbtn():
    return Form(hx_post=btnres, hx_target='#dest')(
        Input(placeholder='Who do you want to greet?', id="nm"),
        Button('Greet'),
        P(id='dest')
    )

In [24]:
p(testbtn)

# Executing Code cells

In [25]:
from toolslm.shell import get_shell


In [26]:
sh = get_shell()

In [27]:
sh.run_cell('1+1')

<ExecutionResult object at 115a10ef0, execution_count=None error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 115a1db50, raw_cell="1+1" transformed_cell="1+1
" store_history=False silent=False shell_futures=True cell_id=None> result=2>

In [28]:
@rt
def execcode(code:str): return P(f"{code}")

In [29]:
def ex(c):
    res = sh.run_cell(c)
    return res.result if res.result is not None else res.stdout


In [30]:
ex("""1 + 10""")

11

In [31]:
ex("""print('Hello, World')""")

'Hello, World\n'

In [32]:
@rt
def runcode():
    return Form(hx_post=execcode, hx_target='#dest', hx_swap='afterbegin')(
        Input(placeholder='Code', id="code"),
        Button('Run code'),
        Div(id='dest')
    )

# Building solveit in solveit

### Resuable compoment deinfition

In [33]:
def mk_comp(comp, dflt_cls):
    def _f(*args, cls='', **kwargs): return comp(*args, cls=f'{dflt_cls} {cls}', **kwargs)
    return _f


In [34]:
mk_comp('Button', 'btn')
mk_comp('Input', 'input')

<function __main__.mk_comp.<locals>._f(*args, cls='', **kwargs)>

In [35]:
p(Button('Test button', cls='btn-primary'))

### Executing input in python

In [36]:
p(Input(type="text", placeholder="Type here"))

In [37]:
@rt
def getinp(): return Input(type='text', placeholder='Type here')

In [38]:
p(getinp)

In [ ]:
# copied from demo notebook
def nameinp(oob=False):
    kw = dict(hx_swap_oob='true') if oob else {}
    return Input(type='text', placeholder='Type here', id='inp', **kw)


@rt
def proc_inp(inp:str): 
    res = ex(inp)
    return P(f"{inp}: {res}"), nameinp(True)

@rt
def testinp():
    return Form(hx_post=proc_inp, hx_target='#test', hx_swap='afterbegin')(
        nameinp(),
        Button('say hi', type='submit'),
        Div(id='test')
    )

In [44]:
p("/testinp")

### Building the UI

In [49]:
from fasthtml.common import html2ft

print(html2ft("""<div class="chat chat-start">
  <div class="chat-bubble chat-bubble-primary">What kind of nonsense is this</div>
</div>
<div class="chat chat-start">
  <div class="chat-bubble chat-bubble-secondary">
    Put me on the Council and not make me a Master!??
  </div>
</div>
<div class="chat chat-start">
  <div class="chat-bubble chat-bubble-accent">
    That's never been done in the history of the Jedi.
  </div>
</div>
<div class="chat chat-start">
  <div class="chat-bubble chat-bubble-neutral">It's insulting!</div>
</div>
<div class="chat chat-end">
  <div class="chat-bubble chat-bubble-info">Calm down, Anakin.</div>
</div>
<div class="chat chat-end">
  <div class="chat-bubble chat-bubble-success">You have been given a great honor.</div>
</div>
<div class="chat chat-end">
  <div class="chat-bubble chat-bubble-warning">To be on the Council at your age.</div>
</div>
<div class="chat chat-end">
  <div class="chat-bubble chat-bubble-error">It's never happened before.</div>
</div>""")) # The daisyUI color chat bubble example

Div(
    Div('What kind of nonsense is this', cls='chat-bubble chat-bubble-primary'),
    cls='chat chat-start'
)

Div(
    Div('Put me on the Council and not make me a Master!??', cls='chat-bubble chat-bubble-secondary'),
    cls='chat chat-start'
)

Div(
    Div("That's never been done in the history of the Jedi.", cls='chat-bubble chat-bubble-accent'),
    cls='chat chat-start'
)

Div(
    Div("It's insulting!", cls='chat-bubble chat-bubble-neutral'),
    cls='chat chat-start'
)

Div(
    Div('Calm down, Anakin.', cls='chat-bubble chat-bubble-info'),
    cls='chat chat-end'
)

Div(
    Div('You have been given a great honor.', cls='chat-bubble chat-bubble-success'),
    cls='chat chat-end'
)

Div(
    Div('To be on the Council at your age.', cls='chat-bubble chat-bubble-warning'),
    cls='chat chat-end'
)

Div(
    Div("It's never happened before.", cls='chat-bubble chat-bubble-error'),
    cls='chat chat-end'
)


In [54]:
p(Div(
    Div(
        Div('Hello World', cls='chat-bubble chat-bubble'),
        cls='chat chat-start'
    ),
    Div(
        Div('Hello.', cls='chat-bubble chat-bubble-primary'),
        cls='chat chat-end'
    ), style="max-width:400px"))

In [59]:
@rt
def proc_code(code:str):
    res = ex(code)
    return (Div(Div(code, cls='chat-bubble chat-bubble-primary'), cls='chat chat-end'),
            Div(Div(str(res), cls='chat-bubble chat-bubble'), cls='chat chat-start') if res else "",
            Input(type='text', placeholder='Enter code...', id='code', hx_swap_oob='true'))

@rt
def chatdemo():
    return Div(
        H1('Code Chat', cls='text-3xl font-bold mb-4'),
        Div(id='chat', cls='mb-4', style='max-width:600px'),
        Form(hx_post=proc_code, hx_target='#chat', hx_swap='beforeend')(
            Input(type='text', placeholder='Enter code...', id='code'),
            Button('Run', type='submit', cls='btn')
        ),
        cls='p-4'
    )

p(chatdemo)

{'apple': 2, 'banana': 4, 'kiwi': 5}

### TODO: Find out how to  pass attributes first to classes to see how it works.

### TODO: Learn more about htmx reference objects. All apart from select and push-to-url. Which aren't very useful

In particular HTMX swap and how its relevant to the above.



## Stop server 
Top stop the server run the below

In [ ]:
server.stop()